1. Takes multiple PDF's as input
2. Chunk it and creates embeddings for them using Huggingface Instructor
3. Save the embeddings in Pinecone free tier index.
4. Extract relevant documents from the vector DB based on the request.

In [37]:
import pdfplumber
path = '05.-Programming-Data-Structure_.pdf'
with pdfplumber.open(path) as pdf:
    text = ""
    for page in pdf.pages:
        text +=  page.extract_text()

In [3]:
print(text)

PPPrrrooogggrrraaammmmmmiiinnnggg
&&&
DDDaaatttaaa SSStttrrruuuccctttuuurrreeesssPublished By:
Physics Wallah
ISBN: 978-93-94342-39-2
Mobile App: Physics Wallah (Available on Play Store)
Website: www.pw.live
Email: support@pw.live
Rights
All rights will be reserved by Publisher. No part of this book may be used or reproduced in any manner
whatsoever without the written permission from author or publisher.
In the interest of student's community:
Circulation of soft copy of Book(s) in PDF or other equivalent format(s) through any social media channels,
emails, etc. or any other channels through mobiles, laptops or desktop is a criminal offence. Anybody
circulating, downloading, storing, soft copy of the book on his device(s) is in breach of Copyright Act. Further
Photocopying of this book or any of its material is also illegal. Do not download or forward in case you come
across any such soft copy material.
Disclaimer
A team of PW experts and faculties with an understanding of the subject

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(separators='\n', chunk_size = 1000, chunk_overlap = 200)
chunks = splitter.split_text(text)

In [5]:
chunks

["PPPrrrooogggrrraaammmmmmiiinnnggg\n&&&\nDDDaaatttaaa SSStttrrruuuccctttuuurrreeesssPublished By:\nPhysics Wallah\nISBN: 978-93-94342-39-2\nMobile App: Physics Wallah (Available on Play Store)\nWebsite: www.pw.live\nEmail: support@pw.live\nRights\nAll rights will be reserved by Publisher. No part of this book may be used or reproduced in any manner\nwhatsoever without the written permission from author or publisher.\nIn the interest of student's community:\nCirculation of soft copy of Book(s) in PDF or other equivalent format(s) through any social media channels,\nemails, etc. or any other channels through mobiles, laptops or desktop is a criminal offence. Anybody\ncirculating, downloading, storing, soft copy of the book on his device(s) is in breach of Copyright Act. Further\nPhotocopying of this book or any of its material is also illegal. Do not download or forward in case you come\nacross any such soft copy material.\nDisclaimer",
 'Photocopying of this book or any of its material

In [6]:
len(chunks[0])

928

In [7]:
print(chunks[0])

PPPrrrooogggrrraaammmmmmiiinnnggg
&&&
DDDaaatttaaa SSStttrrruuuccctttuuurrreeesssPublished By:
Physics Wallah
ISBN: 978-93-94342-39-2
Mobile App: Physics Wallah (Available on Play Store)
Website: www.pw.live
Email: support@pw.live
Rights
All rights will be reserved by Publisher. No part of this book may be used or reproduced in any manner
whatsoever without the written permission from author or publisher.
In the interest of student's community:
Circulation of soft copy of Book(s) in PDF or other equivalent format(s) through any social media channels,
emails, etc. or any other channels through mobiles, laptops or desktop is a criminal offence. Anybody
circulating, downloading, storing, soft copy of the book on his device(s) is in breach of Copyright Act. Further
Photocopying of this book or any of its material is also illegal. Do not download or forward in case you come
across any such soft copy material.
Disclaimer


In [8]:
# %pip install pinecone

In [9]:
import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv('PINECONE_API_KEY')
# print(api_key)

In [10]:
from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(api_key=api_key)

created a free index on pinecone with index name: pdfs.

going to use huggingface instructor model. so set the dimension size of the vector
to 768.

choose the metric to cosine

In [11]:
pc.list_indexes()

{'indexes': [{'deletion_protection': 'disabled',
              'dimension': 768,
              'host': 'pdfs-d53unta.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'pdfs',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [13]:
# %pip install --upgrade langchain


In [8]:
# !pip install InstructorEmbedding==1.0.1
# !pip install sentence-transformers


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [12]:
hf_token = os.getenv('HUGGINGFACE_TOKEN')
# hf_token

In [6]:
# from pydantic import BaseModel


# from langchain_huggingface import HuggingFaceEmbeddings
# from dotenv import load_dotenv
# import os

# # Load environment variables
# load_dotenv()

# # Initialize embeddings with BGE model
# embeddings = HuggingFaceEmbeddings(
#     model_name='BAAI/bge-m3',
#     model_kwargs={'device': 'cpu'},
#     encode_kwargs={'normalize_embeddings': True}
# )

In [ ]:
# from langchain_community.embeddings import HuggingFaceInstructEmbeddings

# model_name = "hkunlp/instructor-large"
# model_kwargs = {
#     'device': 'cpu'
#     # 'use_auth_token': hf_token  # This is the proper way to pass the token
# }
# encode_kwargs = {'normalize_embeddings': True}
# hf = HuggingFaceInstructEmbeddings(
#     model_name=model_name,
#     model_kwargs=model_kwargs,
#     encode_kwargs=encode_kwargs
# )

In [ ]:
# from langchain_community.embeddings import HuggingFaceInstructEmbeddings
# from InstructorEmbedding import INSTRUCTOR

# instructembeddings= HuggingFaceInstructEmbeddings(model_name = "hkunlp/instructor-xl")

# # instructembeddings= HuggingFaceInstructEmbeddings(model_name = "hkunlp/instructor-xl",
# # model_kwargs = {'device': 'cpu'}, encode_kwargs = {'normalize_embeddings': True})

In [17]:
# embedding_model = instructembeddings

NameError: name 'instructembeddings' is not defined

In [ ]:
# from InstructorEmbedding import INSTRUCTOR
# model = INSTRUCTOR('hkunlp/instructor-large')

In [11]:
index_name = 'pdfs'
index = pc.Index(index_name)

In [13]:
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [16]:
# %pip install langchain-pinecone

  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.0.83
    Uninstalling langsmith-0.0.83:
      Successfully uninstalled langsmith-0.0.83
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.19
    Uninstalling langchain-core-0.1.19:
      Successfully uninstalled langchain-core-0.1.19
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.1.2 requires langchain-core<0.2,>=0.1.14, but you have langchain-core 0.3.13 which is incompatible.
langchain 0.1.2 requires langsmith<0.0.84,>=0.0.83, but you have langsmith 0.1.137 which is incompatible.
langchain-cohere 0.1.9 requires langchain-core<0.3,>=0.2.2, but you have langchain-core 0.3.13 which is incompatible.
langchain-community 0.0.18 requires langchain-core<0.2,>=0.1.19, but you have langchain-core 0.3.13 which is incompatible.
langchain-co

In [1]:
# from langchain_pinecone import PineconeVectorStore

# index_name = 'pdfs'

# vectorstore_from_docs = PineconeVectorStore.from_documents(
#         chunks,
#         index_name=index_name,
#         embedding=embedding_model
#     )

# def load_documents(embedding_model, chunks):
#     vectorstore = index.upsert(documents = chunks,  embedding_model = embedding_model, index_name=index_name)
#     return  vectorstore

# vectorstore = load_documents(instructembeddings, chunks=chunks)

/home/jampanasasank/Documents/GenAI/genaienv/lib/python3.10/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [20]:
from langchain_community.vectorstores import Pinecone
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
import pinecone
from pinecone import Pinecone, ServerlessSpec
import os

# Initialize Pinecone
Pinecone(
    api_key=os.getenv('PINECONE_API_KEY')
    # ,environment=os.getenv('PINECONE_ENV')
)

# Your embeddings setup
model_name = "hkunlp/instructor-large"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceInstructEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)


load INSTRUCTOR_Transformer
max_seq_length  512


In [25]:
from langchain_pinecone import PineconeVectorStore
from langchain.schema import Document
documents = [Document(page_content=chunk) for chunk in chunks]


In [24]:
type(chunks)

list

In [26]:
documents

[Document(metadata={}, page_content="PPPrrrooogggrrraaammmmmmiiinnnggg\n&&&\nDDDaaatttaaa SSStttrrruuuccctttuuurrreeesssPublished By:\nPhysics Wallah\nISBN: 978-93-94342-39-2\nMobile App: Physics Wallah (Available on Play Store)\nWebsite: www.pw.live\nEmail: support@pw.live\nRights\nAll rights will be reserved by Publisher. No part of this book may be used or reproduced in any manner\nwhatsoever without the written permission from author or publisher.\nIn the interest of student's community:\nCirculation of soft copy of Book(s) in PDF or other equivalent format(s) through any social media channels,\nemails, etc. or any other channels through mobiles, laptops or desktop is a criminal offence. Anybody\ncirculating, downloading, storing, soft copy of the book on his device(s) is in breach of Copyright Act. Further\nPhotocopying of this book or any of its material is also illegal. Do not download or forward in case you come\nacross any such soft copy material.\nDisclaimer"),
 Document(meta

In [28]:

# Create vector store
index_name = "pdfs"
# pinecone_index = pc.Index(index_name)

vectorstore = PineconeVectorStore.from_documents(
    documents=documents,
    embedding=hf,
    index_name='pdfs'
)

after loading the documents to Pinecone, we can retrieve the relevant documents for the given request using cosine similarity

In [38]:
query = input("Enter query: ")


matching_docs = vectorstore.similarity_search(query, k=2)

In [39]:
query

'explain functions in C'

In [40]:
matching_docs

[Document(id='290d2b1b-94c2-444e-b10c-a53e53bbffbc', metadata={}, page_content='} }\n• Using function poitnter we are able to pass a function as an argurment to other function and can also be returned from\nfunction.\nArray of function pointer\nint (*ptr[3])(int, int)\nPtr is an array of 3 pointers to a function that takes 2 arguments and returns an integer.\n\uf072\uf072\uf072\nGATE WALLAH COMPUTER SCIENCE & INFORMATION TECHNOLOGY HANDBOOK 5.22C Programming\n5\nS\nTRINGS\n5.1 Strings\n• Sequence of characters terminated by a null character ‘\\0’.\nSyntax:\nchar str_name[size];\n• Initializing a string in c\n➢ char str[] = “Pankaj”;\nHere, str is internally a pointer (holds the address of first character)\n➢ char str[20] = “Pankaj”;\nHere, we predefine the size as 20 but we should always take one extra space along with size of string.\ni.e atleast 7 size must be there to store “Pankaj” (6+1)\n➢ char str[7] = {‘P’, ‘a’, ‘n’, ‘k’, ‘a’, ‘j’, ‘\\0’};\nmust set the end character as ‘\\0’\n➢

In [41]:
context = ""
for doc in matching_docs:
    context += doc.page_content

In [42]:
print(context)

} }
• Using function poitnter we are able to pass a function as an argurment to other function and can also be returned from
function.
Array of function pointer
int (*ptr[3])(int, int)
Ptr is an array of 3 pointers to a function that takes 2 arguments and returns an integer.

GATE WALLAH COMPUTER SCIENCE & INFORMATION TECHNOLOGY HANDBOOK 5.22C Programming
5
S
TRINGS
5.1 Strings
• Sequence of characters terminated by a null character ‘\0’.
Syntax:
char str_name[size];
• Initializing a string in c
➢ char str[] = “Pankaj”;
Here, str is internally a pointer (holds the address of first character)
➢ char str[20] = “Pankaj”;
Here, we predefine the size as 20 but we should always take one extra space along with size of string.
i.e atleast 7 size must be there to store “Pankaj” (6+1)
➢ char str[7] = {‘P’, ‘a’, ‘n’, ‘k’, ‘a’, ‘j’, ‘\0’};
must set the end character as ‘\0’
➢ chat str[] = {‘p’, ‘a’, ‘n’, ‘k’, ‘a’, ‘j’, ‘\0’};
5.1.1 Reading a string
• scanf():calls g() i.e., compiler will search